# Stage 2: Synthesis
This notebook uses Gemini to transform raw news into a viral video script.

In [1]:
import json
import os
import time
import google.generativeai as genai

# Load Environment
def load_env(env_path="../.env"):
    if os.path.exists(env_path):
        with open(env_path, "r") as f:
            for line in f:
                if "=" in line:
                    key, val = line.strip().split("=", 1)
                    os.environ[key] = val

load_env()
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY not found!")

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

C:\Users\dynos\AppData\Local\Temp\ipykernel_26572\662274811.py:4: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
def generate_script(news_item):
    prompt = f"""
    You are a viral video producer. Transform the following news into a 30-second engaging video script.
    News Title: {news_item['title']}
    Content: {news_item['content'][:1500]} ...

    Requirements:
    1. 5 sentences max. Engaging, hype, but factual.
    2. For each sentence, provide a visual prompt for an AI image generator.
    3. Suggest a background music mood.
    
    Output strictly valid JSON with this schema:
    {{
      "music_mood": "string",
      "segments": [
        {{"text": "sentence 1", "image_prompt": "visual description"}},
        ...
      ]
    }}
    """
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
             response = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
             return json.loads(response.text)
        except Exception as e:
             print(f"Attempt {attempt+1} failed: {e}")
             time.sleep(2 ** attempt)
    return None

# Load Data
input_path = "../data/raw_news.json"
if not os.path.exists(input_path):
    print("No raw news found")
    news_data = []
else:
    with open(input_path, "r", encoding="utf-8") as f:
        news_data = json.load(f)

# Process first item mainly for demo
scripts = []
if news_data:
    # taking the first one for the pipeline demo
    item = news_data[0]
    print(f"Generating script for: {item['title']} with gemini-pro")
    script = generate_script(item)
    if script:
        scripts.append(script)
        print(f"Generated script!")
    else:
        # Fallback manual script if API fails entirely (to not block the demo)
        print("Generating fallback script due to API failure")
        scripts.append({
            "music_mood": "Tech/Suspense",
            "segments": [
                {"text": "DeepSeek has released R1, slashing AI costs globally.", "image_prompt": "AI robot with price tag"},
                {"text": "This new model rivals the giants like OpenAI.", "image_prompt": "Clash of tech titans"},
                {"text": "Tech stocks are shaking as costs plummet.", "image_prompt": "Stock market graph falling"},
                {"text": "Developers can now run powerful AI on consumer hardware.", "image_prompt": "Laptop with glowing AI code"},
                {"text": "The future of AI is becoming accessible to everyone.", "image_prompt": "Futuristic diverse crowd using AI"}
            ]
        })

os.makedirs("../data", exist_ok=True)
with open("../data/script.json", "w", encoding="utf-8") as f:
    json.dump(scripts, f, indent=4)
    